In [ ]:
import os
os.chdir("..")

In [ ]:
import polars as pl
from src.data.data_process import DataClean

dc = DataClean()

In [ ]:
dc.dataset()

In [ ]:
dc.wb_data(params=["RL.EST", "CC.EST", "RQ.EST", "PV.EST", "VA.EST", "GE.EST"], year=1996)